# Congratulations and Happy Birthday and Thank You Nick!

# What makes a computing language go fast?
<br>
Is it that it is compiled? <br>
Is it that it is jitted? <br>
Is it that it is vectorized? <br>
Is it that the user declares types? <br>
<br> <br> <br><br> <br> <br>
<br> <br> <br><br> <br> <br>
Julia is fast because of multiple dispatch <br>
Julia is fast because we the designers wanted it to be fast <br>
Julia is fast because all the pieces fit together just right <br>
<br> <br> <br><br> <br> <br>
<br> <br> <br><br> <br> <br>
# What makes an algorithm fast?
Does not waste time with unnecessary computations. <br>
Takes advantage of structure.
<br> <br> <br><br> <br> <br>
<br> <br> <br><br> <br> <br>
### Computers run fast when they can execute without extra baggage
Clocks are the ultimate measure of fast, but .... <br>
it does not help us visualize what fast looks like


In [3]:
f(x)=x+5

f (generic function with 1 method)

In [6]:
# We ask a lot from f(x) ....
f(1.0)

6.0

In [8]:
x=10.0;
@code_native f(x)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[3]
Source line: 1
	push	RBP
	mov	RBP, RSP
	movabs	RAX, 4560312336
Source line: 1
	addsd	XMM0, QWORD PTR [RAX]
	pop	RBP
	ret


Looking at assembly can give a clue. <br>
Some guides if you absolutely must decipher the assembly: (not recommended but if you just have to know) <br>
https://en.wikipedia.org/wiki/X86_instruction_listings <br>
http://docs.oracle.com/cd/E36784_01/pdf/E36859.pdf  <br> <br>
It's not exactly the reason code runs fast, but at least it's the reason it's not slow

In [9]:
g()=f(10)
@code_native g()

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[9]
Source line: 1
	push	RBP
	mov	RBP, RSP
	mov	EAX, 15
Source line: 1
	pop	RBP
	ret


In [10]:
x=10.0
@code_native f(x)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[3]
Source line: 1
	push	RBP
	mov	RBP, RSP
	movabs	RAX, 4560314000
Source line: 1
	addsd	XMM0, QWORD PTR [RAX]
	pop	RBP
	ret


See http://blog.leahhanson.us/julia-introspects.html about Julia's ability to introspect.

## Python's addition is really C code


Python's code must check what kind of type <br>
Handles all promotions and integer semantics at run time <br>
(key point is that julia doesn't do more than it has to).  <br>

Julia is *not* a front end to C, Julia is often Julia all the way down

(code below not meant to be read)

In [6]:
static PyLongObject *
long_normalize(PyLongObject *v)
{
    Py_ssize_t j = Py_ABS(Py_SIZE(v));
    Py_ssize_t i = j;

    while (i > 0 && v->ob_digit[i-1] == 0)
        --i;
    if (i != j)
        Py_SIZE(v) = (Py_SIZE(v) < 0) ? -(i) : i;
    return v;
}

static PyLongObject *
x_add(PyLongObject *a, PyLongObject *b)
{
    Py_ssize_t size_a = Py_ABS(Py_SIZE(a)), size_b = Py_ABS(Py_SIZE(b));
    PyLongObject *z;
    Py_ssize_t i;
    digit carry = 0;

    /* Ensure a is the larger of the two: */
    if (size_a < size_b) {
        { PyLongObject *temp = a; a = b; b = temp; }
        { Py_ssize_t size_temp = size_a;
            size_a = size_b;
            size_b = size_temp; }
    }
    z = _PyLong_New(size_a+1);
    if (z == NULL)
        return NULL;
    for (i = 0; i < size_b; ++i) {
        carry += a->ob_digit[i] + b->ob_digit[i];
        z->ob_digit[i] = carry & PyLong_MASK;
        carry >>= PyLong_SHIFT;
    }
    for (; i < size_a; ++i) {
        carry += a->ob_digit[i];
        z->ob_digit[i] = carry & PyLong_MASK;
        carry >>= PyLong_SHIFT;
    }
    z->ob_digit[i] = carry;
    return long_normalize(z);
}

static PyLongObject *
x_sub(PyLongObject *a, PyLongObject *b)
{
    Py_ssize_t size_a = Py_ABS(Py_SIZE(a)), size_b = Py_ABS(Py_SIZE(b));
    PyLongObject *z;
    Py_ssize_t i;
    int sign = 1;
    digit borrow = 0;

    /* Ensure a is the larger of the two: */
    if (size_a < size_b) {
        sign = -1;
        { PyLongObject *temp = a; a = b; b = temp; }
        { Py_ssize_t size_temp = size_a;
            size_a = size_b;
            size_b = size_temp; }
    }
    else if (size_a == size_b) {
        /* Find highest digit where a and b differ: */
        i = size_a;
        while (--i >= 0 && a->ob_digit[i] == b->ob_digit[i])
            ;
        if (i < 0)
            return (PyLongObject *)PyLong_FromLong(0);
        if (a->ob_digit[i] < b->ob_digit[i]) {
            sign = -1;
            { PyLongObject *temp = a; a = b; b = temp; }
        }
        size_a = size_b = i+1;
    }
    z = _PyLong_New(size_a);
    if (z == NULL)
        return NULL;
    for (i = 0; i < size_b; ++i) {
        /* The following assumes unsigned arithmetic
           works module 2**N for some N>PyLong_SHIFT. */
        borrow = a->ob_digit[i] - b->ob_digit[i] - borrow;
        z->ob_digit[i] = borrow & PyLong_MASK;
        borrow >>= PyLong_SHIFT;
        borrow &= 1; /* Keep only one sign bit */
    }
    for (; i < size_a; ++i) {
        borrow = a->ob_digit[i] - borrow;
        z->ob_digit[i] = borrow & PyLong_MASK;
        borrow >>= PyLong_SHIFT;
        borrow &= 1; /* Keep only one sign bit */
    }
    assert(borrow == 0);
    if (sign < 0) {
        _PyLong_Negate(&z);
        if (z == NULL)
            return NULL;
    }
    return long_normalize(z);
}

static PyObject *
long_add(PyLongObject *a, PyLongObject *b)
{
    PyLongObject *z;

    CHECK_BINOP(a, b);

    if (Py_ABS(Py_SIZE(a)) <= 1 && Py_ABS(Py_SIZE(b)) <= 1) {
        PyObject *result = PyLong_FromLong(MEDIUM_VALUE(a) +
                                          MEDIUM_VALUE(b));
        return result;
    }
    if (Py_SIZE(a) < 0) {
        if (Py_SIZE(b) < 0) {
            z = x_add(a, b);
            if (z != NULL && Py_SIZE(z) != 0)
                Py_SIZE(z) = -(Py_SIZE(z));
        }
        else
            z = x_sub(b, a);
    }
    else {
        if (Py_SIZE(b) < 0)
            z = x_sub(a, b);
        else
            z = x_add(a, b);
    }
    return (PyObject *)z;
}

Py_LOCAL_INLINE(void)
_PyLong_Negate(PyLongObject **x_p)
{
    PyLongObject *x;

    x = (PyLongObject *)*x_p;
    if (Py_REFCNT(x) == 1) {
        Py_SIZE(x) = -Py_SIZE(x);
        return;
    }

    *x_p = (PyLongObject *)PyLong_FromLong(-MEDIUM_VALUE(x));
    Py_DECREF(x);
}

PyObject *
PyLong_FromLong(long ival)
{
    PyLongObject *v;
    unsigned long abs_ival;
    unsigned long t;  /* unsigned so >> doesn't propagate sign bit */
    int ndigits = 0;
    int sign = 1;

    CHECK_SMALL_INT(ival);

    if (ival < 0) {
        /* negate: can't write this as abs_ival = -ival since that
           invokes undefined behaviour when ival is LONG_MIN */
        abs_ival = 0U-(unsigned long)ival;
        sign = -1;
    }
    else {
        abs_ival = (unsigned long)ival;
    }

    /* Fast path for single-digit ints */
    if (!(abs_ival >> PyLong_SHIFT)) {
        v = _PyLong_New(1);
        if (v) {
            Py_SIZE(v) = sign;
            v->ob_digit[0] = Py_SAFE_DOWNCAST(
                abs_ival, unsigned long, digit);
        }
        return (PyObject*)v;
    }

#if PyLong_SHIFT==15
    /* 2 digits */
    if (!(abs_ival >> 2*PyLong_SHIFT)) {
        v = _PyLong_New(2);
        if (v) {
            Py_SIZE(v) = 2*sign;
            v->ob_digit[0] = Py_SAFE_DOWNCAST(
                abs_ival & PyLong_MASK, unsigned long, digit);
            v->ob_digit[1] = Py_SAFE_DOWNCAST(
                  abs_ival >> PyLong_SHIFT, unsigned long, digit);
        }
        return (PyObject*)v;
    }
#endif

    /* Larger numbers: loop to determine number of digits */
    t = abs_ival;
    while (t) {
        ++ndigits;
        t >>= PyLong_SHIFT;
    }
    v = _PyLong_New(ndigits);
    if (v != NULL) {
        digit *p = v->ob_digit;
        Py_SIZE(v) = ndigits*sign;
        t = abs_ival;
        while (t) {
            *p++ = Py_SAFE_DOWNCAST(
                t & PyLong_MASK, unsigned long, digit);
            t >>= PyLong_SHIFT;
        }
    }
    return (PyObject *)v;
}

LoadError: syntax: extra token "PyLongObject" after end of expression
while loading In[6], in expression starting on line 1

## Python Assembler Dump

(not meant to be read)

In [7]:
Dump of assembler code for function long_add:
   0x0000000000079050 <+0>:	sub    $0x38,%rsp
   0x0000000000079054 <+4>:	lea    0x20(%rsp),%rcx
   0x0000000000079059 <+9>:	lea    0x18(%rsp),%rdx
   0x000000000007905e <+14>:	mov    %fs:0x28,%rax
   0x0000000000079067 <+23>:	mov    %rax,0x28(%rsp)
   0x000000000007906c <+28>:	xor    %eax,%eax
   0x000000000007906e <+30>:	callq  0x78ea0 <convert_binop>
   0x0000000000079073 <+35>:	test   %eax,%eax
   0x0000000000079075 <+37>:	je     0x790f0 <long_add+160>
   0x0000000000079077 <+39>:	mov    0x18(%rsp),%rdi
   0x000000000007907c <+44>:	cmpq   $0x0,0x10(%rdi)
   0x0000000000079081 <+49>:	js     0x79100 <long_add+176>
   0x0000000000079083 <+51>:	mov    0x20(%rsp),%rsi
   0x0000000000079088 <+56>:	cmpq   $0x0,0x10(%rsi)
   0x000000000007908d <+61>:	js     0x79140 <long_add+240>
   0x0000000000079093 <+67>:	callq  0x774a0 <x_add>
   0x0000000000079098 <+72>:	mov    0x18(%rsp),%rcx
   0x000000000007909d <+77>:	mov    (%rcx),%rsi
   0x00000000000790a0 <+80>:	lea    -0x1(%rsi),%rdx
   0x00000000000790a4 <+84>:	test   %rdx,%rdx
   0x00000000000790a7 <+87>:	mov    %rdx,(%rcx)
   0x00000000000790aa <+90>:	je     0x79120 <long_add+208>
   0x00000000000790ac <+92>:	mov    0x20(%rsp),%rcx
   0x00000000000790b1 <+97>:	mov    (%rcx),%rsi
   0x00000000000790b4 <+100>:	lea    -0x1(%rsi),%rdx
   0x00000000000790b8 <+104>:	test   %rdx,%rdx
   0x00000000000790bb <+107>:	mov    %rdx,(%rcx)
   0x00000000000790be <+110>:	jne    0x790d6 <long_add+134>
   0x00000000000790c0 <+112>:	mov    0x20(%rsp),%rdi
   0x00000000000790c5 <+117>:	mov    %rax,0x8(%rsp)
   0x00000000000790ca <+122>:	mov    0x8(%rdi),%rdx
   0x00000000000790ce <+126>:	callq  *0x30(%rdx)
   0x00000000000790d1 <+129>:	mov    0x8(%rsp),%rax
   0x00000000000790d6 <+134>:	mov    0x28(%rsp),%rcx
   0x00000000000790db <+139>:	xor    %fs:0x28,%rcx
   0x00000000000790e4 <+148>:	jne    0x7917a <long_add+298>
   0x00000000000790ea <+154>:	add    $0x38,%rsp
   0x00000000000790ee <+158>:	retq   
   0x00000000000790ef <+159>:	nop
   0x00000000000790f0 <+160>:	mov    0x301919(%rip),%rax        # 0x37aa10
   0x00000000000790f7 <+167>:	addq   $0x1,(%rax)
   0x00000000000790fb <+171>:	jmp    0x790d6 <long_add+134>
   0x00000000000790fd <+173>:	nopl   (%rax)
   0x0000000000079100 <+176>:	mov    0x20(%rsp),%rax
   0x0000000000079105 <+181>:	cmpq   $0x0,0x10(%rax)
   0x000000000007910a <+186>:	js     0x79150 <long_add+256>
   0x000000000007910c <+188>:	mov    %rdi,%rsi
   0x000000000007910f <+191>:	mov    %rax,%rdi
   0x0000000000079112 <+194>:	callq  0x78100 <x_sub>
   0x0000000000079117 <+199>:	jmpq   0x79098 <long_add+72>
   0x000000000007911c <+204>:	nopl   0x0(%rax)
   0x0000000000079120 <+208>:	mov    0x18(%rsp),%rdi
   0x0000000000079125 <+213>:	mov    %rax,0x8(%rsp)
   0x000000000007912a <+218>:	mov    0x8(%rdi),%rdx
   0x000000000007912e <+222>:	callq  *0x30(%rdx)
   0x0000000000079131 <+225>:	mov    0x8(%rsp),%rax
   0x0000000000079136 <+230>:	jmpq   0x790ac <long_add+92>
   0x000000000007913b <+235>:	nopl   0x0(%rax,%rax,1)
   0x0000000000079140 <+240>:	callq  0x78100 <x_sub>
   0x0000000000079145 <+245>:	jmpq   0x79098 <long_add+72>
   0x000000000007914a <+250>:	nopw   0x0(%rax,%rax,1)
   0x0000000000079150 <+256>:	mov    %rax,%rsi
   0x0000000000079153 <+259>:	callq  0x774a0 <x_add>
   0x0000000000079158 <+264>:	test   %rax,%rax
   0x000000000007915b <+267>:	je     0x79098 <long_add+72>
   0x0000000000079161 <+273>:	mov    0x10(%rax),%rdx
   0x0000000000079165 <+277>:	test   %rdx,%rdx
   0x0000000000079168 <+280>:	je     0x79098 <long_add+72>
   0x000000000007916e <+286>:	neg    %rdx
   0x0000000000079171 <+289>:	mov    %rdx,0x10(%rax)
   0x0000000000079175 <+293>:	jmpq   0x79098 <long_add+72>
   0x000000000007917a <+298>:	callq  0x3c8b0 <__stack_chk_fail@plt>
End of assembler dump.
Dump of assembler code for function x_add:
   0x00000000000774a0 <+0>:	push   %r13
   0x00000000000774a2 <+2>:	push   %r12
   0x00000000000774a4 <+4>:	mov    %rsi,%r13
   0x00000000000774a7 <+7>:	push   %rbp
   0x00000000000774a8 <+8>:	push   %rbx
   0x00000000000774a9 <+9>:	mov    %rdi,%rbp
   0x00000000000774ac <+12>:	sub    $0x8,%rsp
   0x00000000000774b0 <+16>:	mov    0x10(%rdi),%rax
   0x00000000000774b4 <+20>:	sar    $0x3f,%rax
   0x00000000000774b8 <+24>:	mov    %rax,%r12
   0x00000000000774bb <+27>:	xor    0x10(%rdi),%r12
   0x00000000000774bf <+31>:	sub    %rax,%r12
   0x00000000000774c2 <+34>:	mov    0x10(%rsi),%rax
   0x00000000000774c6 <+38>:	sar    $0x3f,%rax
   0x00000000000774ca <+42>:	mov    %rax,%rbx
   0x00000000000774cd <+45>:	xor    0x10(%rsi),%rbx
   0x00000000000774d1 <+49>:	sub    %rax,%rbx
   0x00000000000774d4 <+52>:	cmp    %rbx,%r12
   0x00000000000774d7 <+55>:	jge    0x774e8 <x_add+72>
   0x00000000000774d9 <+57>:	mov    %r12,%rax
   0x00000000000774dc <+60>:	mov    %rsi,%rbp
   0x00000000000774df <+63>:	mov    %rbx,%r12
   0x00000000000774e2 <+66>:	mov    %rdi,%r13
   0x00000000000774e5 <+69>:	mov    %rax,%rbx
   0x00000000000774e8 <+72>:	lea    0x1(%r12),%rdi
   0x00000000000774ed <+77>:	callq  0x3c280 <_PyLong_New@plt>
   0x00000000000774f2 <+82>:	test   %rax,%rax
   0x00000000000774f5 <+85>:	mov    %rax,%r8
   0x00000000000774f8 <+88>:	je     0x775e8 <x_add+328>
   0x00000000000774fe <+94>:	xor    %edx,%edx
   0x0000000000077500 <+96>:	test   %rbx,%rbx
   0x0000000000077503 <+99>:	je     0x77533 <x_add+147>
   0x0000000000077505 <+101>:	xor    %ecx,%ecx
   0x0000000000077507 <+103>:	nopw   0x0(%rax,%rax,1)
   0x0000000000077510 <+112>:	mov    0x18(%r13,%rcx,4),%edi
   0x0000000000077515 <+117>:	add    0x18(%rbp,%rcx,4),%edi
   0x0000000000077519 <+121>:	add    %edi,%edx
   0x000000000007751b <+123>:	mov    %edx,%eax
   0x000000000007751d <+125>:	shr    $0x1e,%edx
   0x0000000000077520 <+128>:	and    $0x3fffffff,%eax
   0x0000000000077525 <+133>:	mov    %eax,0x18(%r8,%rcx,4)
   0x000000000007752a <+138>:	add    $0x1,%rcx
   0x000000000007752e <+142>:	cmp    %rcx,%rbx
   0x0000000000077531 <+145>:	jne    0x77510 <x_add+112>
   0x0000000000077533 <+147>:	cmp    %rbx,%r12
   0x0000000000077536 <+150>:	jle    0x775db <x_add+315>
   0x000000000007753c <+156>:	mov    %rbx,%rcx
   0x000000000007753f <+159>:	nop
   0x0000000000077540 <+160>:	add    0x18(%rbp,%rcx,4),%edx
   0x0000000000077544 <+164>:	mov    %edx,%eax
   0x0000000000077546 <+166>:	shr    $0x1e,%edx
   0x0000000000077549 <+169>:	and    $0x3fffffff,%eax
   0x000000000007754e <+174>:	mov    %eax,0x18(%r8,%rcx,4)
   0x0000000000077553 <+179>:	add    $0x1,%rcx
   0x0000000000077557 <+183>:	cmp    %rcx,%r12
   0x000000000007755a <+186>:	jne    0x77540 <x_add+160>
   0x000000000007755c <+188>:	mov    %edx,0x18(%r8,%r12,4)
   0x0000000000077561 <+193>:	mov    0x10(%r8),%rsi
   0x0000000000077565 <+197>:	mov    %rsi,%rax
   0x0000000000077568 <+200>:	sar    $0x3f,%rax
   0x000000000007756c <+204>:	mov    %rax,%rcx
   0x000000000007756f <+207>:	xor    %rsi,%rcx
   0x0000000000077572 <+210>:	sub    %rax,%rcx
   0x0000000000077575 <+213>:	je     0x775e3 <x_add+323>
   0x0000000000077577 <+215>:	mov    0x14(%r8,%rcx,4),%r9d
   0x000000000007757c <+220>:	lea    -0x1(%rcx),%rdx
   0x0000000000077580 <+224>:	test   %r9d,%r9d
   0x0000000000077583 <+227>:	je     0x775a0 <x_add+256>
   0x0000000000077585 <+229>:	jmp    0x775e3 <x_add+323>
   0x0000000000077587 <+231>:	nopw   0x0(%rax,%rax,1)
   0x0000000000077590 <+240>:	lea    -0x1(%rdx),%rax
   0x0000000000077594 <+244>:	mov    0x18(%r8,%rax,4),%edi
   0x0000000000077599 <+249>:	test   %edi,%edi
   0x000000000007759b <+251>:	jne    0x775c8 <x_add+296>
   0x000000000007759d <+253>:	mov    %rax,%rdx
   0x00000000000775a0 <+256>:	test   %rdx,%rdx
   0x00000000000775a3 <+259>:	jne    0x77590 <x_add+240>
   0x00000000000775a5 <+261>:	mov    %rdx,%rax
   0x00000000000775a8 <+264>:	neg    %rax
   0x00000000000775ab <+267>:	test   %rsi,%rsi
   0x00000000000775ae <+270>:	cmovs  %rax,%rdx
   0x00000000000775b2 <+274>:	mov    %r8,%rax
   0x00000000000775b5 <+277>:	mov    %rdx,0x10(%r8)
   0x00000000000775b9 <+281>:	add    $0x8,%rsp
   0x00000000000775bd <+285>:	pop    %rbx
   0x00000000000775be <+286>:	pop    %rbp
   0x00000000000775bf <+287>:	pop    %r12
   0x00000000000775c1 <+289>:	pop    %r13
   0x00000000000775c3 <+291>:	retq   
   0x00000000000775c4 <+292>:	nopl   0x0(%rax)
   0x00000000000775c8 <+296>:	cmp    %rdx,%rcx
   0x00000000000775cb <+299>:	mov    %r8,%rax
   0x00000000000775ce <+302>:	jne    0x775a5 <x_add+261>
   0x00000000000775d0 <+304>:	add    $0x8,%rsp
   0x00000000000775d4 <+308>:	pop    %rbx
   0x00000000000775d5 <+309>:	pop    %rbp
   0x00000000000775d6 <+310>:	pop    %r12
   0x00000000000775d8 <+312>:	pop    %r13
   0x00000000000775da <+314>:	retq   
   0x00000000000775db <+315>:	mov    %rbx,%r12
   0x00000000000775de <+318>:	jmpq   0x7755c <x_add+188>
   0x00000000000775e3 <+323>:	mov    %r8,%rax
   0x00000000000775e6 <+326>:	jmp    0x775b9 <x_add+281>
   0x00000000000775e8 <+328>:	xor    %eax,%eax
   0x00000000000775ea <+330>:	jmp    0x775b9 <x_add+281>
End of assembler dump.
Dump of assembler code for function x_sub:
   0x0000000000078100 <+0>:	push   %r14
   0x0000000000078102 <+2>:	push   %r13
   0x0000000000078104 <+4>:	mov    %rsi,%r14
   0x0000000000078107 <+7>:	push   %r12
   0x0000000000078109 <+9>:	push   %rbp
   0x000000000007810a <+10>:	mov    %rdi,%r12
   0x000000000007810d <+13>:	push   %rbx
   0x000000000007810e <+14>:	mov    0x10(%rdi),%rax
   0x0000000000078112 <+18>:	sar    $0x3f,%rax
   0x0000000000078116 <+22>:	mov    %rax,%r13
   0x0000000000078119 <+25>:	xor    0x10(%rdi),%r13
   0x000000000007811d <+29>:	sub    %rax,%r13
   0x0000000000078120 <+32>:	mov    0x10(%rsi),%rax
   0x0000000000078124 <+36>:	sar    $0x3f,%rax
   0x0000000000078128 <+40>:	mov    %rax,%rbp
   0x000000000007812b <+43>:	xor    0x10(%rsi),%rbp
   0x000000000007812f <+47>:	sub    %rax,%rbp
   0x0000000000078132 <+50>:	cmp    %rbp,%r13
   0x0000000000078135 <+53>:	jl     0x78278 <x_sub+376>
   0x000000000007813b <+59>:	mov    $0x1,%ebx
   0x0000000000078140 <+64>:	je     0x7823f <x_sub+319>
   0x0000000000078146 <+70>:	mov    %r13,%rdi
   0x0000000000078149 <+73>:	callq  0x3c280 <_PyLong_New@plt>
   0x000000000007814e <+78>:	test   %rax,%rax
   0x0000000000078151 <+81>:	mov    %rax,%r8
   0x0000000000078154 <+84>:	je     0x782d0 <x_sub+464>
   0x000000000007815a <+90>:	xor    %edx,%edx
   0x000000000007815c <+92>:	test   %rbp,%rbp
   0x000000000007815f <+95>:	jle    0x782c9 <x_sub+457>
   0x0000000000078165 <+101>:	xor    %edi,%edi
   0x0000000000078167 <+103>:	nopw   0x0(%rax,%rax,1)
   0x0000000000078170 <+112>:	mov    0x18(%r12,%rdi,4),%ecx
   0x0000000000078175 <+117>:	sub    0x18(%r14,%rdi,4),%ecx
   0x000000000007817a <+122>:	sub    %edx,%ecx
   0x000000000007817c <+124>:	mov    %ecx,%eax
   0x000000000007817e <+126>:	mov    %ecx,%edx
   0x0000000000078180 <+128>:	and    $0x3fffffff,%eax
   0x0000000000078185 <+133>:	shr    $0x1e,%edx
   0x0000000000078188 <+136>:	mov    %eax,0x18(%r8,%rdi,4)
   0x000000000007818d <+141>:	add    $0x1,%rdi
   0x0000000000078191 <+145>:	and    $0x1,%edx
   0x0000000000078194 <+148>:	cmp    %rdi,%rbp
   0x0000000000078197 <+151>:	jne    0x78170 <x_sub+112>
   0x0000000000078199 <+153>:	cmp    %rbp,%r13
   0x000000000007819c <+156>:	jle    0x781c2 <x_sub+194>
   0x000000000007819e <+158>:	xchg   %ax,%ax
   0x00000000000781a0 <+160>:	mov    0x18(%r12,%rbp,4),%eax
   0x00000000000781a5 <+165>:	sub    %edx,%eax
   0x00000000000781a7 <+167>:	mov    %eax,%edx
   0x00000000000781a9 <+169>:	and    $0x3fffffff,%eax
   0x00000000000781ae <+174>:	shr    $0x1e,%edx
   0x00000000000781b1 <+177>:	mov    %eax,0x18(%r8,%rbp,4)
   0x00000000000781b6 <+182>:	add    $0x1,%rbp
   0x00000000000781ba <+186>:	and    $0x1,%edx
   0x00000000000781bd <+189>:	cmp    %rbp,%r13
   0x00000000000781c0 <+192>:	jne    0x781a0 <x_sub+160>
   0x00000000000781c2 <+194>:	cmp    $0xffffffff,%ebx
   0x00000000000781c5 <+197>:	je     0x78298 <x_sub+408>
   0x00000000000781cb <+203>:	mov    0x10(%r8),%rsi
   0x00000000000781cf <+207>:	mov    %rsi,%rax
   0x00000000000781d2 <+210>:	sar    $0x3f,%rax
   0x00000000000781d6 <+214>:	mov    %rax,%rcx
   0x00000000000781d9 <+217>:	xor    %rsi,%rcx
   0x00000000000781dc <+220>:	sub    %rax,%rcx
   0x00000000000781df <+223>:	je     0x7821e <x_sub+286>
   0x00000000000781e1 <+225>:	mov    0x14(%r8,%rcx,4),%r9d
   0x00000000000781e6 <+230>:	lea    -0x1(%rcx),%rdx
   0x00000000000781ea <+234>:	test   %r9d,%r9d
   0x00000000000781ed <+237>:	je     0x78208 <x_sub+264>
   0x00000000000781ef <+239>:	jmp    0x7821e <x_sub+286>
   0x00000000000781f1 <+241>:	nopl   0x0(%rax)
   0x00000000000781f8 <+248>:	lea    -0x1(%rdx),%rax
   0x00000000000781fc <+252>:	mov    0x18(%r8,%rax,4),%edi
   0x0000000000078201 <+257>:	test   %edi,%edi
   0x0000000000078203 <+259>:	jne    0x78260 <x_sub+352>
   0x0000000000078205 <+261>:	mov    %rax,%rdx
   0x0000000000078208 <+264>:	test   %rdx,%rdx
   0x000000000007820b <+267>:	jne    0x781f8 <x_sub+248>
   0x000000000007820d <+269>:	mov    %rdx,%rax
   0x0000000000078210 <+272>:	neg    %rax
   0x0000000000078213 <+275>:	test   %rsi,%rsi
   0x0000000000078216 <+278>:	cmovs  %rax,%rdx
   0x000000000007821a <+282>:	mov    %rdx,0x10(%r8)
   0x000000000007821e <+286>:	mov    %r8,%rax
   0x0000000000078221 <+289>:	pop    %rbx
   0x0000000000078222 <+290>:	pop    %rbp
   0x0000000000078223 <+291>:	pop    %r12
   0x0000000000078225 <+293>:	pop    %r13
   0x0000000000078227 <+295>:	pop    %r14
   0x0000000000078229 <+297>:	retq   
   0x000000000007822a <+298>:	nopw   0x0(%rax,%rax,1)
   0x0000000000078230 <+304>:	mov    0x18(%r14,%rax,4),%ebx
   0x0000000000078235 <+309>:	cmp    %ebx,0x18(%r12,%rax,4)
   0x000000000007823a <+314>:	jne    0x782a8 <x_sub+424>
   0x000000000007823c <+316>:	mov    %rax,%r13
   0x000000000007823f <+319>:	lea    -0x1(%r13),%rax
   0x0000000000078243 <+323>:	cmp    $0xffffffffffffffff,%rax
   0x0000000000078247 <+327>:	jne    0x78230 <x_sub+304>
   0x0000000000078249 <+329>:	pop    %rbx
   0x000000000007824a <+330>:	pop    %rbp
   0x000000000007824b <+331>:	pop    %r12
   0x000000000007824d <+333>:	pop    %r13
   0x000000000007824f <+335>:	pop    %r14
   0x0000000000078251 <+337>:	xor    %edi,%edi
   0x0000000000078253 <+339>:	jmpq   0x3c280 <_PyLong_New@plt>
   0x0000000000078258 <+344>:	nopl   0x0(%rax,%rax,1)
   0x0000000000078260 <+352>:	cmp    %rdx,%rcx
   0x0000000000078263 <+355>:	mov    %r8,%rax
   0x0000000000078266 <+358>:	jne    0x7820d <x_sub+269>
   0x0000000000078268 <+360>:	pop    %rbx
   0x0000000000078269 <+361>:	pop    %rbp
   0x000000000007826a <+362>:	pop    %r12
   0x000000000007826c <+364>:	pop    %r13
   0x000000000007826e <+366>:	pop    %r14
   0x0000000000078270 <+368>:	retq   
   0x0000000000078271 <+369>:	nopl   0x0(%rax)
   0x0000000000078278 <+376>:	mov    %r13,%rax
   0x000000000007827b <+379>:	mov    %rsi,%r12
   0x000000000007827e <+382>:	mov    %rbp,%r13
   0x0000000000078281 <+385>:	mov    %rdi,%r14
   0x0000000000078284 <+388>:	mov    %rax,%rbp
   0x0000000000078287 <+391>:	mov    $0xffffffff,%ebx
   0x000000000007828c <+396>:	jmpq   0x78146 <x_sub+70>
   0x0000000000078291 <+401>:	nopl   0x0(%rax)
   0x0000000000078298 <+408>:	mov    0x10(%r8),%rsi
   0x000000000007829c <+412>:	neg    %rsi
   0x000000000007829f <+415>:	mov    %rsi,0x10(%r8)
   0x00000000000782a3 <+419>:	jmpq   0x781cf <x_sub+207>
   0x00000000000782a8 <+424>:	mov    %r13,%rbp
   0x00000000000782ab <+427>:	mov    $0x1,%ebx
   0x00000000000782b0 <+432>:	jae    0x78146 <x_sub+70>
   0x00000000000782b6 <+438>:	mov    %r12,%rax
   0x00000000000782b9 <+441>:	mov    $0xffffffff,%ebx
   0x00000000000782be <+446>:	mov    %r14,%r12
   0x00000000000782c1 <+449>:	mov    %rax,%r14
   0x00000000000782c4 <+452>:	jmpq   0x78146 <x_sub+70>
   0x00000000000782c9 <+457>:	xor    %ebp,%ebp
   0x00000000000782cb <+459>:	jmpq   0x78199 <x_sub+153>
   0x00000000000782d0 <+464>:	xor    %eax,%eax
   0x00000000000782d2 <+466>:	jmpq   0x78221 <x_sub+289>
End of assembler dump.
Dump of assembler code for function convert_binop:
   0x0000000000078ea0 <+0>:	sub    $0x28,%rsp
   0x0000000000078ea4 <+4>:	mov    0x8(%rdi),%rax
   0x0000000000078ea8 <+8>:	mov    0xa8(%rax),%rax
   0x0000000000078eaf <+15>:	test   $0x1000000,%eax
   0x0000000000078eb4 <+20>:	je     0x78ef0 <convert_binop+80>
   0x0000000000078eb6 <+22>:	mov    %rdi,(%rdx)
   0x0000000000078eb9 <+25>:	addq   $0x1,(%rdi)
   0x0000000000078ebd <+29>:	mov    0x8(%rsi),%rax
   0x0000000000078ec1 <+33>:	mov    0xa8(%rax),%rax
   0x0000000000078ec8 <+40>:	test   $0x1000000,%eax
   0x0000000000078ecd <+45>:	jne    0x78f28 <convert_binop+136>
   0x0000000000078ecf <+47>:	test   $0x800000,%eax
   0x0000000000078ed4 <+52>:	jne    0x78f50 <convert_binop+176>
   0x0000000000078ed6 <+54>:	mov    (%rdx),%rcx
   0x0000000000078ed9 <+57>:	subq   $0x1,(%rcx)
   0x0000000000078edd <+61>:	je     0x78f40 <convert_binop+160>
   0x0000000000078edf <+63>:	xor    %eax,%eax
   0x0000000000078ee1 <+65>:	add    $0x28,%rsp
   0x0000000000078ee5 <+69>:	retq   
   0x0000000000078ee6 <+70>:	nopw   %cs:0x0(%rax,%rax,1)
   0x0000000000078ef0 <+80>:	test   $0x800000,%eax
   0x0000000000078ef5 <+85>:	je     0x78edf <convert_binop+63>
   0x0000000000078ef7 <+87>:	mov    0x10(%rdi),%rdi
   0x0000000000078efb <+91>:	mov    %rcx,0x18(%rsp)
   0x0000000000078f00 <+96>:	mov    %rdx,0x10(%rsp)
   0x0000000000078f05 <+101>:	mov    %rsi,0x8(%rsp)
   0x0000000000078f0a <+106>:	callq  0x3d2c0 <PyLong_FromLong@plt>
   0x0000000000078f0f <+111>:	mov    0x10(%rsp),%rdx
   0x0000000000078f14 <+116>:	mov    0x18(%rsp),%rcx
   0x0000000000078f19 <+121>:	mov    0x8(%rsp),%rsi
   0x0000000000078f1e <+126>:	mov    %rax,(%rdx)
   0x0000000000078f21 <+129>:	jmp    0x78ebd <convert_binop+29>
   0x0000000000078f23 <+131>:	nopl   0x0(%rax,%rax,1)
   0x0000000000078f28 <+136>:	mov    %rsi,(%rcx)
   0x0000000000078f2b <+139>:	mov    $0x1,%eax
   0x0000000000078f30 <+144>:	addq   $0x1,(%rsi)
   0x0000000000078f34 <+148>:	add    $0x28,%rsp
   0x0000000000078f38 <+152>:	retq   
   0x0000000000078f39 <+153>:	nopl   0x0(%rax)
   0x0000000000078f40 <+160>:	mov    (%rdx),%rdi
   0x0000000000078f43 <+163>:	mov    0x8(%rdi),%rax
   0x0000000000078f47 <+167>:	callq  *0x30(%rax)
   0x0000000000078f4a <+170>:	jmp    0x78edf <convert_binop+63>
   0x0000000000078f4c <+172>:	nopl   0x0(%rax)
   0x0000000000078f50 <+176>:	mov    0x10(%rsi),%rdi
   0x0000000000078f54 <+180>:	mov    %rcx,0x8(%rsp)
   0x0000000000078f59 <+185>:	callq  0x3d2c0 <PyLong_FromLong@plt>
   0x0000000000078f5e <+190>:	mov    0x8(%rsp),%rcx
   0x0000000000078f63 <+195>:	mov    %rax,(%rcx)
   0x0000000000078f66 <+198>:	mov    $0x1,%eax
   0x0000000000078f6b <+203>:	jmpq   0x78ee1 <convert_binop+65>
End of assembler dump.
Dump of assembler code for function PyLong_FromLong:
   0x0000000000078de0 <+0>:	push   %r13
   0x0000000000078de2 <+2>:	push   %r12
   0x0000000000078de4 <+4>:	push   %rbp
   0x0000000000078de5 <+5>:	push   %rbx
   0x0000000000078de6 <+6>:	sub    $0x8,%rsp
   0x0000000000078dea <+10>:	test   %rdi,%rdi
   0x0000000000078ded <+13>:	js     0x78e60 <PyLong_FromLong+128>
   0x0000000000078def <+15>:	xor    %r12d,%r12d
   0x0000000000078df2 <+18>:	test   %rdi,%rdi
   0x0000000000078df5 <+21>:	mov    %rdi,%rbx
   0x0000000000078df8 <+24>:	je     0x78e6e <PyLong_FromLong+142>
   0x0000000000078dfa <+26>:	mov    %rbx,%rax
   0x0000000000078dfd <+29>:	xor    %ebp,%ebp
   0x0000000000078dff <+31>:	nop
   0x0000000000078e00 <+32>:	shr    $0x1e,%rax
   0x0000000000078e04 <+36>:	add    $0x1,%ebp
   0x0000000000078e07 <+39>:	test   %rax,%rax
   0x0000000000078e0a <+42>:	jne    0x78e00 <PyLong_FromLong+32>
   0x0000000000078e0c <+44>:	movslq %ebp,%r13
   0x0000000000078e0f <+47>:	mov    %r13,%rdi
   0x0000000000078e12 <+50>:	callq  0x3c280 <_PyLong_New@plt>
   0x0000000000078e17 <+55>:	test   %rax,%rax
   0x0000000000078e1a <+58>:	je     0x78e48 <PyLong_FromLong+104>
   0x0000000000078e1c <+60>:	test   %r12d,%r12d
   0x0000000000078e1f <+63>:	lea    0x18(%rax),%rdx
   0x0000000000078e23 <+67>:	jne    0x78e58 <PyLong_FromLong+120>
   0x0000000000078e25 <+69>:	test   %rbx,%rbx
   0x0000000000078e28 <+72>:	mov    %r13,0x10(%rax)
   0x0000000000078e2c <+76>:	je     0x78e48 <PyLong_FromLong+104>
   0x0000000000078e2e <+78>:	xchg   %ax,%ax
   0x0000000000078e30 <+80>:	mov    %ebx,%ecx
   0x0000000000078e32 <+82>:	shr    $0x1e,%rbx
   0x0000000000078e36 <+86>:	add    $0x4,%rdx
   0x0000000000078e3a <+90>:	and    $0x3fffffff,%ecx
   0x0000000000078e40 <+96>:	test   %rbx,%rbx
   0x0000000000078e43 <+99>:	mov    %ecx,-0x4(%rdx)
   0x0000000000078e46 <+102>:	jne    0x78e30 <PyLong_FromLong+80>
   0x0000000000078e48 <+104>:	add    $0x8,%rsp
   0x0000000000078e4c <+108>:	pop    %rbx
   0x0000000000078e4d <+109>:	pop    %rbp
   0x0000000000078e4e <+110>:	pop    %r12
   0x0000000000078e50 <+112>:	pop    %r13
   0x0000000000078e52 <+114>:	retq   
   0x0000000000078e53 <+115>:	nopl   0x0(%rax,%rax,1)
   0x0000000000078e58 <+120>:	neg    %ebp
   0x0000000000078e5a <+122>:	movslq %ebp,%r13
   0x0000000000078e5d <+125>:	jmp    0x78e25 <PyLong_FromLong+69>
   0x0000000000078e5f <+127>:	nop
   0x0000000000078e60 <+128>:	neg    %rdi
   0x0000000000078e63 <+131>:	mov    $0x1,%r12d
   0x0000000000078e69 <+137>:	mov    %rdi,%rbx
   0x0000000000078e6c <+140>:	jmp    0x78dfa <PyLong_FromLong+26>
   0x0000000000078e6e <+142>:	xor    %r13d,%r13d
   0x0000000000078e71 <+145>:	xor    %ebp,%ebp
   0x0000000000078e73 <+147>:	jmp    0x78e0f <PyLong_FromLong+47>
End of assembler dump.

LoadError: syntax: extra token "of" after end of expression
while loading In[7], in expression starting on line 1

## Does Julia eliminate all this complexity?
No, the complexity is inherent to computing.  Any language may have to manage ints, floats, rationals, bignums, etc. <br>

Any numerical language must manage sparse and dense matrices and many other structures.
Julia organizes the complexity so that at execution time there is no "baggage." <br>

# <center <center> Demmel/Dongarra <br>
<img width=800 src=demmel.png>  <align="left" ><br>


Any Julia user can "lookup" the complexity any time. It's not hidden. <br>
Any Julia user can "drill down"

In [8]:
methods(+)

# 117 methods for generic function "+":
+(x::Bool) at bool.jl:36
+(x::Bool,y::Bool) at bool.jl:39
+(y::FloatingPoint,x::Bool) at bool.jl:49
+(A::BitArray{N},B::BitArray{N}) at bitarray.jl:848
+(A::Union(DenseArray{Bool,N},SubArray{Bool,N,A<:DenseArray{T,N},I<:(Union(Int64,Range{Int64})...,)}),B::Union(DenseArray{Bool,N},SubArray{Bool,N,A<:DenseArray{T,N},I<:(Union(Int64,Range{Int64})...,)})) at array.jl:801
+{S,T}(A::Union(DenseArray{S,N},SubArray{S,N,A<:DenseArray{T,N},I<:(Union(Int64,Range{Int64})...,)}),B::Union(DenseArray{T,N},SubArray{T,N,A<:DenseArray{T,N},I<:(Union(Int64,Range{Int64})...,)})) at array.jl:723
+{T<:Union(Int8,Int32,Int16)}(x::T<:Union(Int8,Int32,Int16),y::T<:Union(Int8,Int32,Int16)) at int.jl:16
+{T<:Union(Uint32,Uint16,Uint8)}(x::T<:Union(Uint32,Uint16,Uint8),y::T<:Union(Uint32,Uint16,Uint8)) at int.jl:20
+(x::Int64,y::Int64) at int.jl:33
+(x::Uint64,y::Uint64) at int.jl:34
+(x::Int128,y::Int128) at int.jl:35
+(x::Uint128,y::Uint128) at int.jl:36
+(x::Float32,y::Float32) at float.jl:119
+(x::Float64,y::Float64) at float.jl:120
+(z::Complex{T<:Real},w::Complex{T<:Real}) at complex.jl:110
+(x::Real,z::Complex{T<:Real}) at complex.jl:120
+(z::Complex{T<:Real},x::Real) at complex.jl:121
+(x::Rational{T<:Integer},y::Rational{T<:Integer}) at rational.jl:113
+(x::Char,y::Char) at char.jl:23
+(x::Char,y::Integer) at char.jl:26
+(x::Integer,y::Char) at char.jl:27
+(a::Float16,b::Float16) at float16.jl:132
+(x::BigInt,y::BigInt) at gmp.jl:195
+(a::BigInt,b::BigInt,c::BigInt) at gmp.jl:218
+(a::BigInt,b::BigInt,c::BigInt,d::BigInt) at gmp.jl:224
+(a::BigInt,b::BigInt,c::BigInt,d::BigInt,e::BigInt) at gmp.jl:231
+(x::BigInt,c::Union(Uint32,Uint64,Uint16,Uint8)) at gmp.jl:243
+(c::Union(Uint32,Uint64,Uint16,Uint8),x::BigInt) at gmp.jl:247
+(x::BigInt,c::Union(Int64,Int8,Int32,Int16)) at gmp.jl:259
+(c::Union(Int64,Int8,Int32,Int16),x::BigInt) at gmp.jl:260
+(x::BigFloat,y::BigFloat) at mpfr.jl:149
+(x::BigFloat,c::Union(Uint32,Uint64,Uint16,Uint8)) at mpfr.jl:156
+(c::Union(Uint32,Uint64,Uint16,Uint8),x::BigFloat) at mpfr.jl:160
+(x::BigFloat,c::Union(Int64,Int8,Int32,Int16)) at mpfr.jl:164
+(c::Union(Int64,Int8,Int32,Int16),x::BigFloat) at mpfr.jl:168
+(x::BigFloat,c::Union(Float64,Float32,Float16)) at mpfr.jl:172
+(c::Union(Float64,Float32,Float16),x::BigFloat) at mpfr.jl:176
+(x::BigFloat,c::BigInt) at mpfr.jl:180
+(c::BigInt,x::BigFloat) at mpfr.jl:184
+(a::BigFloat,b::BigFloat,c::BigFloat) at mpfr.jl:255
+(a::BigFloat,b::BigFloat,c::BigFloat,d::BigFloat) at mpfr.jl:261
+(a::BigFloat,b::BigFloat,c::BigFloat,d::BigFloat,e::BigFloat) at mpfr.jl:268
+(x::MathConst{sym},y::MathConst{sym}) at constants.jl:23
+{T<:Number}(x::T<:Number,y::T<:Number) at promotion.jl:188
+{T<:FloatingPoint}(x::Bool,y::T<:FloatingPoint) at bool.jl:46
+(x::Number,y::Number) at promotion.jl:158
+(x::Integer,y::Ptr{T}) at pointer.jl:68
+(x::Bool,A::AbstractArray{Bool,N}) at array.jl:771
+(x::Number) at operators.jl:71
+(r1::OrdinalRange{T,S},r2::OrdinalRange{T,S}) at operators.jl:325
+{T<:FloatingPoint}(r1::FloatRange{T<:FloatingPoint},r2::FloatRange{T<:FloatingPoint}) at operators.jl:331
+(r1::FloatRange{T<:FloatingPoint},r2::FloatRange{T<:FloatingPoint}) at operators.jl:348
+(r1::FloatRange{T<:FloatingPoint},r2::OrdinalRange{T,S}) at operators.jl:349
+(r1::OrdinalRange{T,S},r2::FloatRange{T<:FloatingPoint}) at operators.jl:350
+(x::Ptr{T},y::Integer) at pointer.jl:66
+{S,T<:Real}(A::Union(DenseArray{S,N},SubArray{S,N,A<:DenseArray{T,N},I<:(Union(Int64,Range{Int64})...,)}),B::Range{T<:Real}) at array.jl:731
+{S<:Real,T}(A::Range{S<:Real},B::Union(DenseArray{T,N},SubArray{T,N,A<:DenseArray{T,N},I<:(Union(Int64,Range{Int64})...,)})) at array.jl:740
+(A::AbstractArray{Bool,N},x::Bool) at array.jl:770
+{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti},B::SparseMatrixCSC{Tv,Ti}) at sparse/sparsematrix.jl:554
+{TvA,TiA,TvB,TiB}(A::SparseMatrixCSC{TvA,TiA},B::SparseMatrixCSC{TvB,TiB}) at sparse/sparsematrix.jl:546
+(A::SparseMatrixCSC{Tv,Ti<:Integer},B::Array{T,N}

In [14]:
x=1.0; y=1;

@which(x+y)

+(x::Number,y::Number) at promotion.jl:158

In [10]:
x=int64(1);y=int64(1)
@code_native(x+y)

	.section	__TEXT,__text,regular,pure_instructions
Filename: int.jl
Source line: 33
	push	RBP
	mov	RBP, RSP
Source line: 33
	add	RDI, RSI
	mov	RAX, RDI
	pop	RBP
	ret


In [11]:
x=int128(1);y=int128(1)
x+y
@code_native(x+y)

	.section	__TEXT,__text,regular,pure_instructions
Filename: int.jl
Source line: 35
	push	RBP
	mov	RBP, RSP
Source line: 35
	add	RDI, RDX
	adc	RSI, RCX
	mov	RAX, RDI
	mov	RDX, RSI
	pop	RBP
	ret


In [12]:
x=1; y=1.0;
x+y
@code_native(x+y)

	.section	__TEXT,__text,regular,pure_instructions
Filename: promotion.jl
Source line: 158
	push	RBP
	mov	RBP, RSP
Source line: 158
	cvtsi2sd	XMM1, RDI
	addsd	XMM1, XMM0
	movaps	XMM0, XMM1
	pop	RBP
	ret


In [13]:
x=1.0; y=1;
@code_native(x+y)

	.section	__TEXT,__text,regular,pure_instructions
Filename: promotion.jl
Source line: 158
	push	RBP
	mov	RBP, RSP
Source line: 158
	cvtsi2sd	XMM1, RDI
	addsd	XMM1, XMM0
	movaps	XMM0, XMM1
	pop	RBP
	ret


In [14]:
x=1.0; y=1.0;
@code_native(x+y)

LoadError: no method found for the specified argument types
while loading In[14], in expression starting on line 2

## Desired Goal might be viewed as:  Get efficient assembler
Traditional Approaches: <br>
Compiler:  Preprocess all you like, taking your time precomputing <br>
Interpreted Languages: Concern with human time primarily, and maybe run time secondary.  Big blockbuster algorithms written in compiled languages and may amortize calling costs. <br>
<br>
Julia: Assumption that codes are more than just a few BLAS3 calls, and needs to strike a different balance
between human and computer.
<br>
<br>  Note Julia programmers rarely look at assembler.... but you can!!!


## This begs the question of how one gets efficient assembler.

In [15]:
1+2   # Hitting LLVM bottom

3

In [16]:
Core.Intrinsics.add_int

(intrinsic function #3)

In [17]:
Core.Intrinsics.add_float

(intrinsic function #12)

In [18]:
x=5; y=4
@code_llvm(x+y)


define i64 @"julia_+_20999"(i64, i64) {
top:
  %2 = add i64 %1, %0, !dbg !2946
  ret i64 %2, !dbg !2946
}


In [19]:
x=5.0; y=4.0
@code_llvm(x+y)


declare double @"julia_+2705"(double, double)


In [20]:
x=5; y=4.0;
x+y
@code_llvm(x+y)


define double @"julia_+_21000"(i64, double) {
top:
  %2 = sitofp i64 %0 to double, !dbg !2949
  %3 = fadd double %2, %1, !dbg !2949
  ret double %3, !dbg !2949
}


In [21]:
@code_typed(x+y)

1-element Array{Any,1}:
 :($(Expr(:lambda, {:x,:y}, {{},{{:x,Int64,0},{:y,Float64,0}},{}}, :(begin  # promotion.jl, line 158:
        return box(Float64,add_float(box(Float64,sitofp(Float64,x::Int64))::Float64,y::Float64))::Float64
    end::Float64))))

In [22]:
x=5.0;y=1.0;
@code_llvm(x+y)


declare double @"julia_+2705"(double, double)


In [23]:
@code_typed(x+y)

1-element Array{Any,1}:
 :($(Expr(:lambda, {:x,:y}, {{},{{:x,Float64,0},{:y,Float64,0}},{}}, :(begin  # float.jl, line 120:
        return box(Float64,add_float(x::Float64,y::Float64))::Float64
    end::Float64))))

In [24]:
x=1.0; y=2.0;
@which(x+y)

+(x::Float64,y::Float64) at float.jl:120

Why does the user not have to declare x and y to be floats and ints? <br>
This is the beauty of the combo of type inference and multiple dispatch.

# <img width=500 src=http://1.bp.blogspot.com/-Zi5Pwx9zH0M/UdlvM0PMDEI/AAAAAAAAPE8/9-neT6ybTTM/s1600/Tropical+beaches.jpg>  <align="left" ><br><center <center> Tropical Numbers  <br>

References <br>
[1] <a href="http://www.maths.manchester.ac.uk/~mheil/LMS_Prospects/pdfs/Tisseur.pdf"> Slides by Françoise Tisseur (Dec 2012)</a> <br>
[2] <a href="http://www.mit.edu/~kepner/pubs/JuliaSemiring_HPEC2013_Paper.pdf"> Novel Algebras for Advanced Analytics in Julia </a>
<br>
Comment: Inspired by Françoise's lovely talk at Householder IX (2014) <br>

In [1]:
#Define your favorite plus, and times
⊕(x,y) = max(x,y)       # Put your favoring semiring plus here
⊗(x,y) = x+y            # Put your favorite semiring times here
                        # Best if ⊗ distributes over ⊕

⊗ (generic function with 1 method)

In [2]:
importall Base
immutable TNum{T} <: Number
    val::T
end

# immutable means if you execute this twice, it will warn you it's not cool

In [3]:
TNum(1)

TNum{Int64}(1)

In [4]:
TNum(2.0)

TNum{Float64}(2.0)

In [5]:
TNum(1//2)
inv([1//(i+j) for i=1:3,j=1:3  ])

3x3 Array{Rational{Int64},2}:
   72//1  -240//1   180//1
 -240//1   900//1  -720//1
  180//1  -720//1   600//1

In [6]:
TNum([1 2;4 5])

TNum{Array{Int64,2}}(2x2 Array{Int64,2}:
 1  2
 4  5)

In [7]:
importall Base
Base.zero{T}(::Type{TNum{T}}) = TNum(zero(T))
 Base.one{T}(::Type{TNum{T}}) = TNum(one(T))
+(a::TNum, b::TNum) = TNum(a.val ⊕ b.val)
*(a::TNum, b::TNum) = TNum(a.val ⊗ b.val)
Base.show(io::IO, a::TNum) = print(io, a.val)

show (generic function with 96 methods)

In [10]:
x=TNum(5)
y=TNum(6)
@show x +  y
@show x^10

x + y => 6
x^10 => 50


50

In [11]:
@code_native(x+y)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[7]
Source line: 4
	push	RBP
	mov	RBP, RSP
	cmp	RSI, RDI
Source line: 4
	cmovge	RDI, RSI
	mov	RAX, RDI
	pop	RBP
	ret


In [12]:
@code_native(x*y)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[7]
Source line: 5
	push	RBP
	mov	RBP, RSP
Source line: 5
	add	RDI, RSI
	mov	RAX, RDI
	pop	RBP
	ret


In [35]:
x=TNum(1.0); y=TNum(3.0);
x*y
@code_native(x+y)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[31]
Source line: 6
	push	RBP
	mov	RBP, RSP
	ucomisd	XMM1, XMM0
Source line: 6
	setbe	AL
	ucomisd	XMM0, XMM0
	setnp	CL
	test	CL, AL
	jne	4
	movapd	XMM0, XMM1
	pop	RBP
	ret


In [30]:
z=TNum(NaN)

NaN

In [31]:
x*z

NaN

In [32]:
x+z

5.0

In [39]:
# TNum(A::Array) = map(TNum, A)            # Let's have an array of tropicals, rather than a tropical array
# conj(a::TNum)=a                          # Transposes should just be like reals

In [33]:
A=[1 2;3 4]
B=[100 2;43 5]
TNum(A) * TNum(B)              # This did what we said, but not what we want.  

2x2 Array{Int64,2}:
 101  4
  46  9

In [41]:
# What we really want TNum of an array to be is rather an array of TNums
TNum(A::Array) = map(TNum, A)

TNum{T} (constructor with 2 methods)

In [42]:
methods(TNum)

# 2 methods for generic function "TNum":
TNum(A::Array{T,N}) at In[41]:2
TNum{T}(val::T)

In [77]:
A=TNum(A)
B=TNum(B)

2x2 Array{TNum{Int64},2}:
 100  2
  43  5

In [78]:
A*B

2x2 Array{TNum{Int64},2}:
 101  7
 103  9

In [79]:
# max(1+100,2+43) etc.

In [46]:
A^5

2x2 Array{TNum{Int64},2}:
 17  18
 19  20

In [34]:
A'*A

2x2 Array{Int64,2}:
 10  14
 14  20

In [48]:
A=TNum([1 2 3 5])
A'*A

4x4 Array{TNum{Int64},2}:
 2  3  4   6
 3  4  5   7
 4  5  6   8
 6  7  8  10

In [49]:
A'*A

4x4 Array{TNum{Int64},2}:
 2  3  4   6
 3  4  5   7
 4  5  6   8
 6  7  8  10

In [35]:
kron(A,B)

4x4 Array{Int64,2}:
 100   2  200   4
  43   5   86  10
 300   6  400   8
 129  15  172  20

In [36]:
x=TNum(1.0im)
x * complex(5.0,0)



LoadError: no promotion exists for TNum{Complex{Float64}} and Complex{Float64}
while loading In[36], in expression starting on line 2

## Code Reuse

Notice that we do not write separate tropical algebras for real, complex, float, and int.  
We also do not write new matrix multiplies, nor integer powers.  This is the beauty of generic
programming.  

## Multiple Dispatch

In [37]:
promote_rule{T,U}(::Type{TNum{T}},::Type{U})  =  TNum{promote_type(T,U)}
convert{T}(::Type{TNum{T}}, x::TNum) = TNum(convert(T,x.val));
convert{T,U}(::Type{TNum{T}}, x::U) = TNum(convert(T,x))

convert (generic function with 450 methods)

In [38]:
i = TNum(1.0*im)
5 * i

5.0 + 1.0im

In [84]:
5 * TNum(1.0*im)
TNum(1.0*im) * 5

5.0 + 1.0im

In [85]:
13 + 10*i

13.0 + 0.0im

In [56]:
isless(x::Complex{Float64}, y::Complex{Float64}) =
real(x)<real(y) || real(x)==real(y) && imag(x)<imag(y)

isless (generic function with 19 methods)

In [57]:
13 + 10*i

13.0 + 0.0im

In [58]:
@code_native( TNum(1) * 5.0 )

	.section	__TEXT,__text,regular,pure_instructions
Filename: promotion.jl
Source line: 159
	push	RBP
	mov	RBP, RSP
Source line: 159
	cvtsi2sd	XMM1, RDI
	addsd	XMM1, XMM0
	movaps	XMM0, XMM1
	pop	RBP
	ret


In [59]:
@code_native TNum(3//5) * 2//3

	.section	__TEXT,__text,regular,pure_instructions
Filename: promotion.jl
Source line: 159
	push	RBP
	mov	RBP, RSP
	push	R15
	push	R14
	push	R13
	push	R12
	push	RBX
	push	RAX
	mov	R14, RCX
	mov	R15, RDX
Source line: 159
	movabs	RBX, 4567854832
	call	RBX
	mov	RDI, RAX
	mov	RSI, RDX
	call	RBX
	mov	R13, RAX
	mov	R12, RDX
	mov	RDI, R15
	mov	RSI, R14
	call	RBX
	mov	RDI, RAX
	mov	RSI, RDX
	call	RBX
	mov	RCX, RDX
	movabs	RBX, 4567857312
	mov	RDI, R13
	mov	RSI, R12
	mov	RDX, RAX
	call	RBX
	add	RSP, 8
	pop	RBX
	pop	R12
	pop	R13
	pop	R14
	pop	R15
	pop	RBP
	ret


## Linear Algebra

In [60]:
A=SymTridiagonal([1, 2, 3],[4, 5])

3x3 SymTridiagonal{Int64}:
 1  4  0
 4  2  5
 0  5  3

In [61]:
B=rand(1:5,3,3)

3x3 Array{Int64,2}:
 4  3  4
 2  3  1
 1  4  3

In [62]:
A + B

3x3 Array{Int64,2}:
 5  7  4
 6  5  6
 1  9  6

In [63]:
@which(A+B)   

+(A::SymTridiagonal{T},B::Array{T,2}) at linalg/special.jl:98

If you can see that this high level linear algebra mechansim is the same as the low level type mechanism then you start to appreciate the message of Julia

In [64]:
 A=[1 2 3
1 2 1
1 0 1
1 0 -1]

4x3 Array{Int64,2}:
 1  2   3
 1  2   1
 1  0   1
 1  0  -1

In [65]:
QR = qrfact(A)
Q  = QR[:Q]
R  = QR[:R]

3x3 Array{Float64,2}:
 -2.0  -2.0  -2.0
  0.0  -2.0  -2.0
  0.0   0.0  -2.0

In [66]:
QR \ rand(4)

3-element Array{Float64,1}:
 0.631251 
 0.0123648
 0.0108939

In [67]:
Q * [1,0,0,0]

4-element Array{Float64,1}:
 -0.5
 -0.5
 -0.5
 -0.5

In [68]:
Q * [1,0,0]

4-element Array{Float64,1}:
 -0.5
 -0.5
 -0.5
 -0.5

In [69]:
Q * [1,0,0,0,0]

LoadError: DimensionMismatch("")
while loading In[69], in expression starting on line 1